<a href="https://colab.research.google.com/github/DikshantBadawadagi/Image-Binary-Matrix/blob/main/BPCS%20%2B%202DLogistic-Maps%20%2B%20DNA%20Encoding%20%2B%203DLogistic%20Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
import random

def logistic_map_2d(x0, y0, r, shape):
    x, y = x0, y0
    sequence = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = r * x * (1 - x)
            y = r * y * (1 - y)
            sequence[i, j] = (x + y) / 2
    return sequence

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

def arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (x + y) % width
                new_y = (x + 2*y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def inverse_arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (2*x - y) % width
                new_y = (-x + y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def encrypt_image(image_array, key, arnold_iterations):
    encrypted = arnold_transform(image_array, arnold_iterations)
    encrypted = encrypted.astype(np.int16)
    if len(image_array.shape) == 3:  # Color image
        for c in range(image_array.shape[2]):
            encrypted[:,:,c] = (encrypted[:,:,c] + (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        encrypted = (encrypted + (key * 256).astype(np.int16)) % 256
    return encrypted.astype(np.uint8)

def decrypt_image(encrypted_array, key, arnold_iterations):
    decrypted = encrypted_array.astype(np.int16)
    if len(encrypted_array.shape) == 3:  # Color image
        for c in range(encrypted_array.shape[2]):
            decrypted[:,:,c] = (decrypted[:,:,c] - (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        decrypted = (decrypted - (key * 256).astype(np.int16)) % 256
    decrypted = inverse_arnold_transform(decrypted.astype(np.uint8), arnold_iterations)
    return decrypted.astype(np.uint8)

def encrypt_image_file(input_path, output_path):
    x0 = random.uniform(0, 1)
    y0 = random.uniform(0, 1)
    r = random.uniform(3.8, 4.0)
    arnold_iterations = random.randint(1, 10)
    image_array = image_to_array(input_path)
    key = logistic_map_2d(x0, y0, r, image_array.shape[:2])
    encrypted_array = encrypt_image(image_array, key, arnold_iterations)
    array_to_image(encrypted_array, output_path)
    return x0, y0, r, arnold_iterations

def decrypt_image_file(encrypted_path, output_path, x0, y0, r, arnold_iterations):
    encrypted_array = image_to_array(encrypted_path)
    key = logistic_map_2d(x0, y0, r, encrypted_array.shape[:2])
    decrypted_array = decrypt_image(encrypted_array, key, arnold_iterations)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
x0, y0, r, arnold_iterations = encrypt_image_file(input_image_path, encrypted_image_path)
print(f"Encryption parameters: x0={x0}, y0={y0}, r={r}, arnold_iterations={arnold_iterations}")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, x0, y0, r, arnold_iterations)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption parameters: x0=0.45873957524167297, y0=0.5292915928689771, r=3.8180785198327727, arnold_iterations=3
Decryption complete.
Original image shape: (225, 225, 3)
Encrypted image shape: (225, 225, 3)
Decrypted image shape: (225, 225, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [166 160 110]
Encrypted image sample: [153 147  97]
Decrypted image sample: [166 160 110]
Mean difference between original and decrypted images: 0.0
Maximum difference between original and decrypted images: 0.0
The decrypted image is identical to the original image.


In [ ]:
import numpy as np
from PIL import Image
import random

def logistic_map_2d(x0, y0, r, shape):
    x, y = x0, y0
    sequence = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = r * x * (1 - x)
            y = r * y * (1 - y)
            sequence[i, j] = (x + y) / 2
    return sequence

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

def arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (x + y) % width
                new_y = (x + 2*y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def inverse_arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (2*x - y) % width
                new_y = (-x + y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

# DNA coding functions
def int_to_dna(n):
    dna_map = {0: 'A', 1: 'T', 2: 'C', 3: 'G'}
    return dna_map[n]

def dna_to_int(dna):
    dna_map = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    return dna_map[dna]

def pixel_to_dna(pixel):
    return ''.join(int_to_dna((pixel >> i) & 3) for i in (6, 4, 2, 0))

def dna_to_pixel(dna):
    return sum(dna_to_int(dna[i]) << (6 - 2*i) for i in range(4))

def dna_encode(array):
    return np.vectorize(pixel_to_dna)(array)

def dna_decode(dna_array):
    return np.vectorize(dna_to_pixel)(dna_array)

def dna_xor(dna1, dna2):
    return ''.join(int_to_dna(dna_to_int(a) ^ dna_to_int(b)) for a, b in zip(dna1, dna2))

def generate_key(shape):
    return np.random.randint(0, 256, shape, dtype=np.uint8)

def encrypt_image(image_array, key, arnold_iterations):
    # Arnold transform
    encrypted = arnold_transform(image_array, arnold_iterations)
    encrypted = encrypted.astype(np.int16)

    # DNA encryption
    if len(encrypted.shape) == 3:  # Color image
        encrypted_dna = np.empty_like(encrypted, dtype=object)
        for c in range(encrypted.shape[2]):
            dna_image = dna_encode(encrypted[:,:,c])
            dna_key = dna_encode(key)
            encrypted_dna[:,:,c] = np.frompyfunc(dna_xor, 2, 1)(dna_image, dna_key)
        encrypted = np.apply_along_axis(lambda x: dna_decode(x.astype(str)), 2, encrypted_dna)
    else:  # Grayscale image
        dna_image = dna_encode(encrypted)
        dna_key = dna_encode(key)
        encrypted_dna = np.frompyfunc(dna_xor, 2, 1)(dna_image, dna_key)
        encrypted = dna_decode(encrypted_dna)

    return encrypted.astype(np.uint8)

def decrypt_image(encrypted_array, key, arnold_iterations):
    # DNA decryption
    if len(encrypted_array.shape) == 3:  # Color image
        decrypted = np.empty_like(encrypted_array, dtype=object)
        for c in range(encrypted_array.shape[2]):
            dna_encrypted = dna_encode(encrypted_array[:,:,c])
            dna_key = dna_encode(key)
            decrypted[:,:,c] = np.frompyfunc(dna_xor, 2, 1)(dna_encrypted, dna_key)
        decrypted = np.apply_along_axis(lambda x: dna_decode(x.astype(str)), 2, decrypted)
    else:  # Grayscale image
        dna_encrypted = dna_encode(encrypted_array)
        dna_key = dna_encode(key)
        decrypted_dna = np.frompyfunc(dna_xor, 2, 1)(dna_encrypted, dna_key)
        decrypted = dna_decode(decrypted_dna)

    # Inverse Arnold transform
    decrypted = inverse_arnold_transform(decrypted.astype(np.uint8), arnold_iterations)
    return decrypted.astype(np.uint8)

def encrypt_image_file(input_path, output_path):
    x0 = random.uniform(0, 1)
    y0 = random.uniform(0, 1)
    r = random.uniform(3.8, 4.0)
    arnold_iterations = random.randint(1, 10)
    image_array = image_to_array(input_path)
    key = logistic_map_2d(x0, y0, r, image_array.shape[:2])
    key_array = (key * 255).astype(np.uint8)
    encrypted_array = encrypt_image(image_array, key_array, arnold_iterations)
    array_to_image(encrypted_array, output_path)
    return x0, y0, r, arnold_iterations


def decrypt_image_file(encrypted_path, output_path, x0, y0, r, arnold_iterations):
    encrypted_array = image_to_array(encrypted_path)
    key = logistic_map_2d(x0, y0, r, encrypted_array.shape[:2])
    key_array = (key * 255).astype(np.uint8)
    decrypted_array = decrypt_image(encrypted_array, key_array, arnold_iterations)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
x0, y0, r, arnold_iterations = encrypt_image_file(input_image_path, encrypted_image_path)
print(f"Encryption parameters: x0={x0}, y0={y0}, r={r}, arnold_iterations={arnold_iterations}")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, x0, y0, r, arnold_iterations)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption parameters: x0=0.8815269802875294, y0=0.9972870928298356, r=3.9900452222167346, arnold_iterations=10
Decryption complete.
Original image shape: (192, 204, 3)
Encrypted image shape: (192, 204, 3)
Decrypted image shape: (192, 204, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [254 254 254]
Encrypted image sample: [54 54 54]
Decrypted image sample: [0 0 0]
Mean difference between original and decrypted images: 97.1538756127451
Maximum difference between original and decrypted images: 255.0
The decrypted image differs from the original image.
Number of different pixels: 117108


BIT PLANE COMPLEXITY SEGEMENTATION

In [ ]:
import numpy as np
from PIL import Image

def image_to_array(image_path):
    """Convert image to NumPy array."""
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    """Convert NumPy array to image and save."""
    print(f"Debug: Array shape before conversion: {array.shape}")
    print(f"Debug: Array dtype: {array.dtype}")

    if array.ndim == 4 and array.shape[3] == 1:
        array = array.squeeze(axis=3)

    if array.ndim == 3 and array.shape[2] == 3:
        image = Image.fromarray(array.astype(np.uint8), mode='RGB')
    elif array.ndim == 2:
        image = Image.fromarray(array.astype(np.uint8), mode='L')
    else:
        raise ValueError(f"Unsupported array shape for image conversion: {array.shape}")
    image.save(output_path)

def split_into_bit_planes(image_array):
    """Split the image array into bit-planes."""
    print(f"Debug: Image array shape before splitting: {image_array.shape}")

    # Ensure the array is 3D (add channel dimension if grayscale)
    if image_array.ndim == 2:
        image_array = image_array[:, :, np.newaxis]

    # Split each color channel into bit planes
    height, width, channels = image_array.shape
    bit_planes = np.unpackbits(image_array, axis=2).reshape(height, width, channels, 8)

    print(f"Debug: Bit planes shape: {bit_planes.shape}")
    return bit_planes

def reconstruct_from_bit_planes(bit_planes):
    """Reconstruct image from bit-planes."""
    print(f"Debug: Bit planes shape before reconstruction: {bit_planes.shape}")

    reconstructed = np.packbits(bit_planes, axis=-1)
    print(f"Debug: Reconstructed shape: {reconstructed.shape}")

    return reconstructed.squeeze(axis=-1)

def encrypt_decrypt_bit_planes(bit_planes, key):
    """Encrypt or decrypt bit-planes by XORing with a key."""
    return bit_planes ^ key[:, :, np.newaxis, np.newaxis]

def generate_random_key(seed, shape):
    """Generate a random key for XOR encryption using a seed."""
    np.random.seed(seed)
    return np.random.randint(0, 2, shape, dtype=np.uint8)

def process_image_bpcs(image_path, output_path, seed=42, encrypt=True):
    """Encrypt or decrypt an image using Bit-Plane Complexity Segmentation (BPCS)."""
    image_array = image_to_array(image_path)
    print(f"Debug: Original image shape: {image_array.shape}")

    bit_planes = split_into_bit_planes(image_array)

    # Generate a random key for encryption based on seed
    key_shape = (bit_planes.shape[0], bit_planes.shape[1])
    key = generate_random_key(seed, key_shape)

    # Encrypt or decrypt the bit-planes
    processed_bit_planes = encrypt_decrypt_bit_planes(bit_planes, key)

    # Reconstruct the processed image
    processed_image = reconstruct_from_bit_planes(processed_bit_planes)
    print(f"Debug: Processed image shape: {processed_image.shape}")

    array_to_image(processed_image, output_path)

def encrypt_image_bpcs(image_path, output_path, seed=42):
    """Encrypt an image using BPCS."""
    process_image_bpcs(image_path, output_path, seed, encrypt=True)

def decrypt_image_bpcs(encrypted_path, output_path, seed=42):
    """Decrypt an image using BPCS."""
    process_image_bpcs(encrypted_path, output_path, seed, encrypt=False)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encrypt the image
encrypt_image_bpcs(input_image_path, encrypted_image_path)

# Decrypt the image
decrypt_image_bpcs(encrypted_image_path, decrypted_image_path)

Debug: Original image shape: (192, 204, 3)
Debug: Image array shape before splitting: (192, 204, 3)
Debug: Bit planes shape: (192, 204, 3, 8)
Debug: Bit planes shape before reconstruction: (192, 204, 3, 8)
Debug: Reconstructed shape: (192, 204, 3, 1)
Debug: Processed image shape: (192, 204, 3)
Debug: Array shape before conversion: (192, 204, 3)
Debug: Array dtype: uint8
Debug: Original image shape: (192, 204, 3)
Debug: Image array shape before splitting: (192, 204, 3)
Debug: Bit planes shape: (192, 204, 3, 8)
Debug: Bit planes shape before reconstruction: (192, 204, 3, 8)
Debug: Reconstructed shape: (192, 204, 3, 1)
Debug: Processed image shape: (192, 204, 3)
Debug: Array shape before conversion: (192, 204, 3)
Debug: Array dtype: uint8


BITPLANE combined with LOGICTIC 2D maps combined with DNA encoding


In [ ]:
import numpy as np
from PIL import Image
from typing import Tuple, List

def image_to_array(image_path: str) -> np.ndarray:
    """Convert image to NumPy array."""
    with Image.open(image_path) as img:
        return np.array(img)

def array_to_image(array: np.ndarray, output_path: str) -> None:
    """Convert NumPy array to image and save."""
    if array.ndim == 3 and array.shape[2] == 3:
        mode = 'RGB'
    elif array.ndim == 2 or (array.ndim == 3 and array.shape[2] == 1):
        array = array.squeeze()
        mode = 'L'
    else:
        raise ValueError(f"Unsupported array shape: {array.shape}")

    Image.fromarray(array.astype(np.uint8), mode=mode).save(output_path)

def split_into_bit_planes(image_array: np.ndarray) -> np.ndarray:
    if image_array.ndim == 2:
        image_array = image_array[:, :, np.newaxis]
    return np.unpackbits(image_array.astype(np.uint8), axis=2).reshape(*image_array.shape, 8)

def reconstruct_from_bit_planes(bit_planes: np.ndarray) -> np.ndarray:
    return np.packbits(bit_planes.astype(np.uint8), axis=-1).squeeze(axis=-1)

def logistic_map(x: float, y: float, a: float, b: float, iterations: int) -> Tuple[float, float]:
    """Generate a 2D logistic map."""
    for _ in range(iterations):
        x_new = (a * x * (1 - x) - b * y * y) % 1
        y_new = (b * x * x + a * y * (1 - y)) % 1
        x, y = x_new, y_new
    return x, y

def generate_logistic_key(seed: int, shape: Tuple[int, int]) -> np.ndarray:
    """Generate a key using the 2D logistic map."""
    np.random.seed(seed)
    a, b = np.random.uniform(3.57, 3.99, size=2)
    x, y = np.random.uniform(0, 1, size=2)

    key = np.zeros(shape, dtype=np.uint8)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x, y = logistic_map(x, y, a, b, 100)
            key[i, j] = int(((x + y) % 1) * 256) % 256
    return key

DNA_MAP = {'00': 'A', '01': 'C', '10': 'G', '11': 'T'}
DNA_MAP_REVERSE = {v: k for k, v in DNA_MAP.items()}

def dna_encode(data: np.ndarray) -> str:
    """Encode data using DNA encoding."""
    binary = ''.join([f'{b:08b}' for b in data.flatten()])
    return ''.join(DNA_MAP[binary[i:i+2]] for i in range(0, len(binary), 2))

def dna_decode(dna_string: str) -> np.ndarray:
    """Decode DNA-encoded data."""
    binary_string = ''.join(DNA_MAP_REVERSE[base] for base in dna_string)
    return np.array([int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8)], dtype=np.uint8)

def encrypt_decrypt_bit_planes_dna(bit_planes: np.ndarray, key: np.ndarray) -> np.ndarray:
    """Encrypt or decrypt bit-planes using XOR operation."""
    # Expand key to match bit_planes shape
    key_expanded = np.repeat(key[:, :, np.newaxis], bit_planes.shape[2], axis=2)
    key_expanded = np.repeat(key_expanded[:, :, :, np.newaxis], bit_planes.shape[3], axis=3)

    # Convert key to binary (0 and 1)
    key_binary = np.unpackbits(key_expanded.astype(np.uint8), axis=-1)[:, :, :, :8]

    # Perform XOR operation
    return np.bitwise_xor(bit_planes, key_binary).astype(np.uint8)

def process_image_bpcs_logistic_dna(image_path: str, output_path: str, seed: int = 42) -> None:
    """Process an image using BPCS with Logistic 2D maps and DNA encoding."""
    image_array = image_to_array(image_path)
    print(f"Original image shape: {image_array.shape}")
    print(f"Original image min/max: {image_array.min()}/{image_array.max()}")

    bit_planes = split_into_bit_planes(image_array)
    print(f"Bit planes shape: {bit_planes.shape}")
    print(f"Bit planes min/max: {bit_planes.min()}/{bit_planes.max()}")

    key = generate_logistic_key(seed, (bit_planes.shape[0], bit_planes.shape[1]))
    print(f"Generated key shape: {key.shape}")
    print(f"Key min/max: {key.min()}/{key.max()}")

    processed_bit_planes = encrypt_decrypt_bit_planes_dna(bit_planes, key)
    print(f"Processed bit planes shape: {processed_bit_planes.shape}")
    print(f"Processed bit planes min/max: {processed_bit_planes.min()}/{processed_bit_planes.max()}")

    processed_image = reconstruct_from_bit_planes(processed_bit_planes)
    print(f"Processed image shape: {processed_image.shape}")
    print(f"Processed image min/max: {processed_image.min()}/{processed_image.max()}")

    array_to_image(processed_image, output_path)

if __name__ == "__main__":
    input_image = "/content/download.jpg"
    encrypted_image = "/content/encrypted.png"
    decrypted_image = "/content/decrypted.png"
    seed = 42

    encrypt_image(input_image, encrypted_image, seed)
    decrypt_image(encrypted_image, decrypted_image, seed)

Encrypting image...
Original image shape: (192, 204, 3)
Original image min/max: 0/255
Bit planes shape: (192, 204, 3, 8)
Bit planes min/max: 0/1
Generated key shape: (192, 204)
Key min/max: 0/255
Processed bit planes shape: (192, 204, 3, 8)
Processed bit planes min/max: 0/1
Processed image shape: (192, 204, 3)
Processed image min/max: 0/255
Encrypted image saved to /content/encrypted.png
Decrypting image...
Original image shape: (192, 204, 3)
Original image min/max: 0/255
Bit planes shape: (192, 204, 3, 8)
Bit planes min/max: 0/1
Generated key shape: (192, 204)
Key min/max: 0/255
Processed bit planes shape: (192, 204, 3, 8)
Processed bit planes min/max: 0/1
Processed image shape: (192, 204, 3)
Processed image min/max: 0/255
Decrypted image saved to /content/decrypted.png


In [ ]:
import numpy as np
from PIL import Image

# Logistic map parameters
def logistic_map_3d(x, y, z, rx, ry, rz, alpha, beta, gamma, iterations):
    x_seq, y_seq, z_seq = [], [], []

    for _ in range(iterations):
        x_next = rx * x * (1 - x) + beta * (y * z)
        y_next = ry * y * (1 - y) + alpha * (x * z)
        z_next = rz * z * (1 - z) + gamma * (x * y)

        x, y, z = x_next, y_next, z_next
        x_seq.append(x)
        y_seq.append(y)
        z_seq.append(z)

    return np.array(x_seq), np.array(y_seq), np.array(z_seq)

# Image encryption using XOR
def encrypt_image(image, logistic_map_params):
    x0, y0, z0, rx, ry, rz, alpha, beta, gamma = logistic_map_params
    pixels = np.array(image)

    # Get the number of pixels (rows * cols) and flatten the image
    flat_pixels = pixels.flatten()
    n_pixels = flat_pixels.size

    # Generate chaotic sequence
    x_seq, y_seq, z_seq = logistic_map_3d(x0, y0, z0, rx, ry, rz, alpha, beta, gamma, n_pixels)

    # Scale chaotic sequences to the range [0, 255]
    chaotic_seq = ((x_seq + y_seq + z_seq) * 255).astype(np.uint8)

    # XOR the pixel values with the chaotic sequence
    encrypted_pixels = np.bitwise_xor(flat_pixels, chaotic_seq)

    # Reshape back to the original image shape
    encrypted_image = encrypted_pixels.reshape(pixels.shape)
    return Image.fromarray(encrypted_image)

# Image decryption (same as encryption process with the same parameters)
def decrypt_image(encrypted_image, logistic_map_params):
    return encrypt_image(encrypted_image, logistic_map_params)

# Main function to load the image, encrypt it, and decrypt it
def main():
    # Load an image
    img = Image.open('/content/download.jpg').convert('L')  # Convert image to grayscale for simplicity

    # Logistic map initial conditions and parameters
    logistic_map_params = (0.1, 0.1, 0.1, 3.9, 3.9, 3.9, 0.5, 0.5, 0.5)

    # Encrypt the image
    encrypted_img = encrypt_image(img, logistic_map_params)
    encrypted_img.save('/content/encrypted_image.png')

    # Decrypt the image
    decrypted_img = decrypt_image(encrypted_img, logistic_map_params)
    decrypted_img.save('/content/decrypted_image.png')

# Run the main function
if __name__ == "__main__":
    main()
